# I. Import thư viện

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import islice
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight
import pickle
from joblib import dump, load

In [17]:
#from google.colab import drive
#drive.mount('/content/drive')

# II. Đọc dữ liệu

In [18]:
#df = pd.read_csv('/content/drive/MyDrive/DS317/df.csv')
df = pd.read_csv('D:\Learning\Semester 1 year 4\DS317\Project\Data\daxuly.csv')

In [19]:
df.head(5)

,mssv,soquyetdinh,ngaycapvb,namsinh,gioitinh,noisinh,lopsh,khoa,hedt,khoahoc,...,tctichluy5,tctichluy6,tc_tichluy_3_5,socc_tienganh,tienganh,canhcao,tc_yeucau,du_dktn,tb_drl,label
0,BE375BAAXPvAibaEXe9JDlHA4z2GHJ3/PVStCxR2,204/QĐ-ĐHCNTT,2019-04-12,1995.0,1.0,Vùng Đông Nam Bộ,KTPM0001,CNPM,CQUI,8.0,...,16.0,NaN,12.0,1.0,Unknown,0.0,153.0,1,77.7,Khá
1,2420ED57XPvAibaEXe/Lh6v1XxTKJa6JLFRUPkLM,675/QĐ-ĐHCNTT,2019-10-11,1995.0,1.0,Vùng Đồng bằng sông Cửu Long,HTTT0001,HTTT,CTTT,8.0,...,10.0,NaN,10.0,1.0,Unknown,1.0,152.0,0,82.1,Rớt
2,83B76C01XPvAibaEXe/lOccskaOiO2K46r7t4qnt,NaN,NaN,1994.0,1.0,Vùng Đồng bằng sông Hồng,KHMT2013,KHMT,CQUI,8.0,...,NaN,NaN,NaN,0.0,0,1.0,140.0,0,35.5,Rớt
3,91F785ABXPvAibaEXe/lOccskaOiO5y4GbVvuRQu,204/QĐ-ĐHCNTT,2019-04-12,1995.0,1.0,Vùng Đông Nam Bộ,HTTT0001,HTTT,CTTT,8.0,...,27.0,0.0,10.0,1.0,TOEIC_LR,0.0,152.0,0,63.5,Rớt
4,007C275DXPvAibaEXe+TFgEDwYNnveOOmOYeYzF6,NaN,NaN,1995.0,1.0,Vùng Đông Nam Bộ,MMTT0001,MMT&TT,CQUI,8.0,...,3.0,NaN,15.0,0.0,0,2.0,148.0,0,77.4,Rớt


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7826 entries, 0 to 7825
Data columns (total 59 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   mssv            7826 non-null   object 
 1   soquyetdinh     1572 non-null   object 
 2   ngaycapvb       1572 non-null   object 
 3   namsinh         7826 non-null   float64
 4   gioitinh        7826 non-null   float64
 5   noisinh         7826 non-null   object 
 6   lopsh           7826 non-null   object 
 7   khoa            7826 non-null   object 
 8   hedt            7826 non-null   object 
 9   khoahoc         7826 non-null   float64
 10  chuyennganh2    7826 non-null   object 
 11  tinhtrang       7826 non-null   float64
 12  diachi_tinhtp   7824 non-null   object 
 13  diemnamhoc1     7798 non-null   float64
 14  diemnamhoc2     7407 non-null   float64
 15  diemnamhoc3     7087 non-null   float64
 16  diemnamhoc4     6788 non-null   float64
 17  diemnamhoc5     3322 non-null   f

In [21]:
cer_dict = {
    'Cambrigde': 1,
    'IELTS': 2,
    'NHAT': 3,
    'PHAP': 4,
    'TOEFL iBT': 5,
    'TOEIC_LR': 6,
    'TOEIC_SW': 7,
    'TOIEC': 8,
    'VNU-EPT': 9,
    'DGNL': 10,
    'Unknown': 11,
    'Multi_cer': 12
}

'''
cer_dict = {
    'Cambrigde': 1,
    'IELTS': 2,
    'TOEFL iBT': 3,
    'TOEIC_LR': 4,
    'TOEIC_SW': 5,
    'TOIEC': 6,
    'VNU-EPT': 7,
    'DGNL': 8
}
'''

"\ncer_dict = {\n    'Cambrigde': 1,\n    'IELTS': 2,\n    'TOEFL iBT': 3,\n    'TOEIC_LR': 4,\n    'TOEIC_SW': 5,\n    'TOIEC': 6,\n    'VNU-EPT': 7,\n    'DGNL': 8\n}\n"

In [22]:
 # Xử lý cột tiếng anh
def cer_encode(row):
  if row['socc_tienganh'] != 0:
    if row['socc_tienganh'] == 1:
      return cer_dict[row['tienganh']]
    else:
      return 9
  else:
    return 0

df['tienganh'] = df.apply(cer_encode, axis=1)

In [23]:
df = df.drop(columns={'mssv', 'ghichu', 'soquyetdinh', 'ngaycapvb', 'lop12_matinh', 'lop12_matruong', 'tb_drl', 'tc_rot_tong', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'lopsh', 'tc_yeucau', 'du_dktn'})
#df = df.drop(columns={'tb_drl', 'tc_rot_tong', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'du_dktn'})
df = df.rename(columns={'label': 'xeploai'})

In [24]:
# Bỏ bớt các điểm và đrl từ năm học thứ 5 trở lên và điền khuyết cho năm học 4
df = df.drop(columns=['diemnamhoc4', 'diemnamhoc5', 'diemnamhoc6', 'drlnam4', 'drlnam5', 'drlnam6', 'sotc_rot4', 'sotc_rot5', 'sotc_rot6', 'sotc_rot7', 'sotc_rot8', 'tctichluy4', 'tctichluy5', 'tctichluy6'])

In [25]:
df.isnull().sum()

namsinh              0
gioitinh             0
noisinh              0
khoa                 0
hedt                 0
khoahoc              0
chuyennganh2         0
tinhtrang            0
diachi_tinhtp        2
diemnamhoc1         28
diemnamhoc2        419
diemnamhoc3        739
diem_3_5          1070
drlnam1             78
drlnam2            419
drlnam3            739
drl_3_5           1009
dien_tt              0
diem_tt              0
sotc_rot1            0
sotc_rot2            0
sotc_rot3            0
rotmon_3_5           0
tctichluy1           8
tctichluy2         397
tctichluy3         719
tc_tichluy_3_5    1061
socc_tienganh        0
tienganh             0
canhcao              0
xeploai              0
dtype: int64

In [26]:
# Thuộc tính tc tích lũy năm 3.5 điền giá trị trung bình, các thuộc tính còn lại điền giá trị trung vị
df['diemnamhoc1'] = df['diemnamhoc1'].fillna(df['diemnamhoc1'].median())
df['diemnamhoc2'] = df['diemnamhoc2'].fillna(df['diemnamhoc2'].median())
df['diemnamhoc3'] = df['diemnamhoc3'].fillna(df['diemnamhoc3'].median())
df['diem_3_5'] = df['diem_3_5'].fillna(df['diem_3_5'].mean())
df['drlnam1'] = df['drlnam1'].fillna(df['drlnam1'].median())
df['drlnam2'] = df['drlnam2'].fillna(df['drlnam2'].median())
df['drlnam3'] = df['drlnam3'].fillna(df['drlnam3'].median())
df['drl_3_5'] = df['drl_3_5'].fillna(df['drl_3_5'].median())
df['tctichluy1'] = df['tctichluy1'].fillna(df['tctichluy1'].median())
df['tctichluy2'] = df['tctichluy2'].fillna(df['tctichluy2'].median())
df['tctichluy3'] = df['tctichluy3'].fillna(df['tctichluy3'].median())
df['tc_tichluy_3_5'] = df['tc_tichluy_3_5'].fillna(df['tc_tichluy_3_5'].median())
df['diachi_tinhtp'] = df['diachi_tinhtp'].fillna(df['diachi_tinhtp'].mode()[0])

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7826 entries, 0 to 7825
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   namsinh         7826 non-null   float64
 1   gioitinh        7826 non-null   float64
 2   noisinh         7826 non-null   object 
 3   khoa            7826 non-null   object 
 4   hedt            7826 non-null   object 
 5   khoahoc         7826 non-null   float64
 6   chuyennganh2    7826 non-null   object 
 7   tinhtrang       7826 non-null   float64
 8   diachi_tinhtp   7826 non-null   object 
 9   diemnamhoc1     7826 non-null   float64
 10  diemnamhoc2     7826 non-null   float64
 11  diemnamhoc3     7826 non-null   float64
 12  diem_3_5        7826 non-null   float64
 13  drlnam1         7826 non-null   float64
 14  drlnam2         7826 non-null   float64
 15  drlnam3         7826 non-null   float64
 16  drl_3_5         7826 non-null   float64
 17  dien_tt         7826 non-null   o

In [28]:
df['xeploai'].value_counts()

Rớt                3038
Chưa tốt nghiệp    2873
Khá                1122
Giỏi                460
Trung bình khá      209
Trung bình          111
Xuất sắc             13
Name: xeploai, dtype: int64

In [29]:
df_1 = df.copy()
df_2 = df.copy()

In [30]:
xl_dict = {
    'Rớt': 0,
    'Chưa tốt nghiệp': 0,
    'Trung bình': 1,
    'Trung bình khá': 2,
    'Khá': 3,
    'Giỏi': 4,
    'Xuất sắc': 5
}

In [31]:
def encode_xl(df):
  return xl_dict[df['xeploai']]

df_1['xeploai'] = df_1.apply(encode_xl, axis=1)

In [32]:
df_2 = df_2[df_2['xeploai'] != 'Chưa tốt nghiệp']

In [33]:
cate_col = df.select_dtypes(include=['object']).columns
cate_col

Index(['noisinh', 'khoa', 'hedt', 'chuyennganh2', 'diachi_tinhtp', 'dien_tt',
       'xeploai'],
      dtype='object')

In [34]:
#Encode các biến categorical
label_encoder = preprocessing.LabelEncoder()

for u in cate_col:
  df_1[u] = label_encoder.fit_transform(df_1[u])
for u in cate_col:
  df_2[u] = label_encoder.fit_transform(df_2[u])

In [35]:
df_1.head(5)

,namsinh,gioitinh,noisinh,khoa,hedt,khoahoc,chuyennganh2,tinhtrang,diachi_tinhtp,diemnamhoc1,...,sotc_rot3,rotmon_3_5,tctichluy1,tctichluy2,tctichluy3,tc_tichluy_3_5,socc_tienganh,tienganh,canhcao,xeploai
0,1995.0,1.0,4,0,2,8.0,6,3.0,3,6.47,...,3.0,0.0,35.0,40.0,36.0,12.0,1.0,11,0.0,3
1,1995.0,1.0,5,1,3,8.0,7,3.0,3,7.72,...,0.0,0.0,35.0,34.0,35.0,10.0,1.0,11,1.0,0
2,1994.0,1.0,6,2,2,8.0,4,5.0,5,5.03,...,0.0,0.0,27.0,28.0,37.0,16.0,0.0,0,1.0,0
3,1995.0,1.0,4,1,3,8.0,7,3.0,3,7.03,...,0.0,0.0,27.0,32.0,35.0,10.0,1.0,6,0.0,0
4,1995.0,1.0,4,5,2,8.0,8,8.0,3,6.63,...,0.0,0.0,43.0,41.0,32.0,15.0,0.0,0,2.0,0


In [36]:
len(df_1), len(df_2)

(7826, 4953)

In [37]:
df_1.to_csv('D:\Learning\Semester 1 year 4\DS317\Project\Data\Preprocessed_Data_IncludeCTN_For_Models.csv', index=False)

In [38]:
df_2.to_csv('D:\Learning\Semester 1 year 4\DS317\Project\Data\Preprocessed_Data_ExcludeCTN_For_Models.csv', index=False)